In [10]:
import covid_feature_extraction
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.statespace.varmax import VARMAX

In [20]:
def VAR_wrapper(input_X, input_y):
    
    train_frac = 0.85
    
    vector_data = pd.concat([input_y.reset_index(drop=True),input_X["total_cases"],input_X["deaths"],input_X["vaccine_allocations"]],axis=1)
    vector_data_train = vector_data.iloc[:int(train_frac*len(vector_data)),:]
    vector_data_test = vector_data.iloc[int(train_frac*len(vector_data)):,:]
    
    vector_data_diff = vector_data_train.diff().dropna()
    
    # check for stationarity after one differencing
    y_stationarity = adfuller(vector_data_diff[input_y.name])
    y_adf, y_pval = y_stationarity[0], y_stationarity[1]

    total_cases_stationarity = adfuller(vector_data_diff["total_cases"])
    total_cases_adf, total_cases_pval = total_cases_stationarity[0], total_cases_stationarity[1]

    death_stationarity = adfuller(vector_data_diff["deaths"])
    death_adf, death_pval = death_stationarity[0], death_stationarity[1]

    vax_stationarity = adfuller(vector_data_diff["vaccine_allocations"])
    vax_adf, vax_pval = vax_stationarity[0], vax_stationarity[1]

    print("Augmented Dickey–Fuller Test Results\n----------------------------------------")
    print("Y-Feature p-value: {}".format(y_pval))
    print("\nTotal Cases per 100k p-value: {}".format(total_cases_pval))
    print("\nDeaths per 100k p-value: {}".format(death_pval))
    print("\nVaccine Allocations p-value: {}".format(vax_pval))
    
    # fit the chosen model and forecast
    model = VARMAX(vector_data_diff,order = (15,3)).fit()
    lag_num = 15
    lagged_data = vector_data_diff.values[-lag_num:]
    
    forecast = model.forecast(y=lagged_data, steps=len(vector_data_test))
    forecast = pd.DataFrame(forecast, index=vector_data.index[-len(vector_data_test):], columns=vector_data.columns)
    
    # undo the differencing
    forecast_copy = forecast.copy()
    cols = vector_data_train.columns
    for col in cols:        
        forecast_copy[str(col)+'_forecast'] = vector_data_train[col].iloc[-1] + forecast_copy[str(col)].cumsum()
    proper_forecast = forecast_copy.iloc[:,3:]
    
    mse = ((proper_forecast[input_y.name+"_forecast"] - vector_data_test[input_y.name])**2).mean()
    print("Mean Squared-Error: {}".format(mse))
    
    fig, ax = plt.subplots(4,1,figsize=(12,11))

    ax[0].plot(dates[:int(train_frac*len(vector_data))],vector_data_train["total_cases"],color='k',lw=1)
    ax[0].plot(dates[int(train_frac*len(vector_data)):],vector_data_test["total_cases"],color='k',lw=1)
    ax[0].plot(dates[int(train_frac*len(vector_data)):],proper_forecast["total_cases_forecast"],color='r',lw=1,linestyle='--')
    ax[0].tick_params(axis='x',bottom=False,labelbottom=False)
    ax[0].set_ylabel("Total Cases per 100k")

    ax[1].plot(dates[:int(train_frac*len(vector_data))],vector_data_train["deaths"],color='k',lw=1)
    ax[1].plot(dates[int(train_frac*len(vector_data)):],vector_data_test["deaths"],color='k',lw=1)
    ax[1].plot(dates[int(train_frac*len(vector_data)):],proper_forecast["deaths_forecast"],color='r',lw=1,linestyle='--')
    ax[1].tick_params(axis='x',bottom=False,labelbottom=False)
    ax[1].set_ylabel("Deaths per 100k")

    ax[2].plot(dates[:int(train_frac*len(vector_data))],vector_data_train["vaccine_allocations"],color='k',lw=1)
    ax[2].plot(dates[int(train_frac*len(vector_data)):],vector_data_test["vaccine_allocations"],color='k',lw=1)
    ax[2].plot(dates[int(train_frac*len(vector_data)):],proper_forecast["vaccine_allocations_forecast"],color='r',lw=1,linestyle='--')
    ax[2].tick_params(axis='x',bottom=False,labelbottom=False)
    ax[2].set_ylabel("Vaccine Allocations")

    ax[3].plot(dates[:int(train_frac*len(vector_data))],vector_data_train[input_y.name],color='k',lw=1)
    ax[3].plot(dates[int(train_frac*len(vector_data)):],vector_data_test[input_y.name],color='k',lw=1)
    ax[3].plot(dates[int(train_frac*len(vector_data)):],proper_forecast[input_y.name+"_forecast"],color='r',lw=1,linestyle='--')
    ax[3].set_ylabel(input_y.name)

    plt.subplots_adjust(hspace = 0.1)
    plt.show()

# Population Not Staying Home

In [21]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Population")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Population")
dates, state_y, state_X, feature_labels = covid_feature_extraction.state_extraction("Population")

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(state_X.shape[0]-len(dose_allocations),0))
state_X = pd.concat([state_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(state_X, state_y)

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Augmented Dickey–Fuller Test Results
----------------------------------------
Y-Feature p-value: 3.614480595724674e-05

Total Cases per 100k p-value: 0.021770471384510975

Deaths per 100k p-value: 1.8324720073792228e-09

Vaccine Allocations p-value: 0.0


C:\Users\Matt\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\Matt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


LinAlgError: 4-th leading minor of the array is not positive definite

In [19]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Population")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Population")
dates, fulton_y, fulton_X, feature_labels = covid_feature_extraction.county_extraction("Fulton","Population")

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(fulton_X.shape[0]-len(dose_allocations),0))
fulton_X = pd.concat([fulton_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(fulton_X, fulton_y)

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Augmented Dickey–Fuller Test Results
----------------------------------------
Y-Feature p-value: 1.7471287286936295e-05

Total Cases per 100k p-value: 0.00014295756046160576

Deaths per 100k p-value: 3.774197756825268e-13

Vaccine Allocations p-value: 0.0


C:\Users\Matt\anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\Matt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


LinAlgError: Schur decomposition solver error.

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Population")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Population")
dates, lowndes_y, lowndes_X, feature_labels = covid_feature_extraction.county_extraction("Lowndes","Population")

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(lowndes_X.shape[0]-len(dose_allocations),0))
lowndes_X = pd.concat([lowndes_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(lowndes_X, lowndes_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Population")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Population")
dates, chatham_y, chatham_X, feature_labels = covid_feature_extraction.county_extraction("Chatham","Population")

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(chatham_X.shape[0]-len(dose_allocations),0))
chatham_X = pd.concat([chatham_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(chatham_X, chatham_y)

# Long Trips

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Long")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Long")
dates, state_y, state_X, feature_labels = covid_feature_extraction.state_extraction("Long")
state_y.name = "Fraction Long Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(state_X.shape[0]-len(dose_allocations),0))
state_X = pd.concat([state_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(state_X, state_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Long")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Long")
dates, fulton_y, fulton_X, feature_labels = covid_feature_extraction.county_extraction("Fulton","Long")
fulton_y.name = "Fraction Long Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(fulton_X.shape[0]-len(dose_allocations),0))
fulton_X = pd.concat([fulton_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(fulton_X, fulton_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Long")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Long")
dates, lowndes_y, lowndes_X, feature_labels = covid_feature_extraction.county_extraction("Lowndes","Long")
lowndes_y.name = "Fraction Long Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(lowndes_X.shape[0]-len(dose_allocations),0))
lowndes_X = pd.concat([lowndes_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(lowndes_X, lowndes_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Long")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Long")
dates, chatham_y, chatham_X, feature_labels = covid_feature_extraction.county_extraction("Chatham","Long")
chatham_y.name = "Fraction Long Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(chatham_X.shape[0]-len(dose_allocations),0))
chatham_X = pd.concat([chatham_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(chatham_X, chatham_y)

# Medium Trips

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Medium")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Medium")
dates, state_y, state_X, feature_labels = covid_feature_extraction.state_extraction("Medium")
state_y.name = "Fraction Medium Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(state_X.shape[0]-len(dose_allocations),0))
state_X = pd.concat([state_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(state_X, state_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Medium")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Medium")
dates, fulton_y, fulton_X, feature_labels = covid_feature_extraction.county_extraction("Fulton","Medium")
fulton_y.name = "Fraction Medium Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(fulton_X.shape[0]-len(dose_allocations),0))
fulton_X = pd.concat([fulton_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(fulton_X, fulton_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Medium")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Medium")
dates, lowndes_y, lowndes_X, feature_labels = covid_feature_extraction.county_extraction("Lowndes","Medium")
lowndes_y.name = "Fraction Medium Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(lowndes_X.shape[0]-len(dose_allocations),0))
lowndes_X = pd.concat([lowndes_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(lowndes_X, lowndes_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Medium")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Medium")
dates, chatham_y, chatham_X, feature_labels = covid_feature_extraction.county_extraction("Chatham","Medium")
chatham_y.name = "Fraction Medium Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(chatham_X.shape[0]-len(dose_allocations),0))
chatham_X = pd.concat([chatham_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(chatham_X, chatham_y)

# Short Trips

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Short")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Short")
dates, state_y, state_X, feature_labels = covid_feature_extraction.state_extraction("Short")
state_y.name = "Fraction Short Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(state_X.shape[0]-len(dose_allocations),0))
state_X = pd.concat([state_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(state_X, state_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Short")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Short")
dates, fulton_y, fulton_X, feature_labels = covid_feature_extraction.county_extraction("Fulton","Short")
fulton_y.name = "Fraction Short Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(fulton_X.shape[0]-len(dose_allocations),0))
fulton_X = pd.concat([fulton_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(fulton_X, fulton_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Short")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Short")
dates, lowndes_y, lowndes_X, feature_labels = covid_feature_extraction.county_extraction("Lowndes","Short")
lowndes_y.name = "Fraction Short Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(lowndes_X.shape[0]-len(dose_allocations),0))
lowndes_X = pd.concat([lowndes_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(lowndes_X, lowndes_y)

In [ ]:
alloc_dates, state_y, first_dose = covid_feature_extraction.first_dose("Georgia","GA","Short")
alloc_dates, state_y, second_dose = covid_feature_extraction.second_dose("Georgia","GA","Short")
dates, chatham_y, chatham_X, feature_labels = covid_feature_extraction.county_extraction("Chatham","Short")
chatham_y.name = "Fraction Short Trips"

dose_allocations = first_dose+second_dose

padded_doses = np.pad(dose_allocations,pad_width=(chatham_X.shape[0]-len(dose_allocations),0))
chatham_X = pd.concat([chatham_X,pd.Series(padded_doses,name="vaccine_allocations")],axis=1)

VAR_wrapper(chatham_X, chatham_y)